In [1]:
import cv2
import numpy as np

In [32]:
image=cv2.imread("Market_image.jpg")
image=cv2.resize(image,(800,800))
image.shape

(800, 800, 3)

In [33]:
height,width,_=image.shape

In [34]:
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [23]:
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

In [24]:
classes=[]
with open("coco.names",'r')as f:
    classes=f.read().splitlines()

In [9]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [35]:
blob=cv2.dnn.blobFromImage(image,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

In [36]:
for each in blob:
    for n,img in enumerate(each):
        cv2.imshow(str(n),img)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
        

In [37]:
net.setInput(blob)
outputLayerNames=net.getUnconnectedOutLayersNames()
layerOutputs=net.forward(outputLayerNames)


In [38]:
boxes=[]
confidences=[]
class_ids=[]

for each in layerOutputs:
    for detection in each:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence>0.5:
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            
            x=int(center_x-(w/2))
            y=int(center_y-(h/2))
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(boxes)
print(confidences)
print(class_ids)
            

[[253, 550, 252, 235], [242, 530, 270, 256], [251, 591, 219, 193], [247, 584, 258, 203], [55, 453, 37, 128], [0, 468, 35, 156], [53, 461, 39, 133], [312, 466, 33, 127], [0, 487, 37, 139], [123, 505, 51, 149], [133, 500, 46, 161], [171, 503, 45, 155], [208, 498, 49, 162], [322, 447, 86, 268], [328, 442, 93, 280], [648, 491, 100, 180], [170, 510, 51, 154], [206, 519, 53, 140], [322, 445, 87, 295], [327, 439, 93, 311], [659, 504, 72, 170], [651, 501, 93, 173], [658, 500, 104, 178], [327, 470, 97, 301], [319, 449, 25, 131]]
[0.9026240706443787, 0.8480573296546936, 0.8521624803543091, 0.9699137806892395, 0.9508848786354065, 0.9684178233146667, 0.6077716946601868, 0.75139981508255, 0.553625762462616, 0.9591202735900879, 0.643412172794342, 0.9867804646492004, 0.9502072334289551, 0.9046753644943237, 0.9893867373466492, 0.9893621802330017, 0.949065089225769, 0.6904006004333496, 0.9950907826423645, 0.9995025396347046, 0.5186010003089905, 0.9865409135818481, 0.5057905912399292, 0.529181182384491,

In [39]:
indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
print(indexes.flatten())

font=cv2.FONT_HERSHEY_PLAIN
colors=np.random.uniform(0,255,size=(len(boxes),3))

for i in indexes.flatten():
    x,y,w,h=boxes[i]
    label=str(classes[class_ids[i]])
    confidence=str(round(confidences[i],2))
    color=colors[i]
    cv2.rectangle(image,(x,y),(x+w,y+h),color,2)
    cv2.putText(image,label+" "+confidence,(x,y+20),font,2,(255,255,255),)
    

[19 15 11  3  5  9  4 12  7]


In [40]:
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()